# QUIZ - 2024-08

In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [3]:
from service.models import Service, ServiceCategory, ServiceProvider, Customer, Appointment
from datetime import date, time, datetime

## PART 1 - CRUD Operations
1.1 สร้างบริการ (`Service`) รายละเอียดตามด้านล่าง และเพิ่มเข้าไปในหมวดหมู่ (`ServiceCategory`) ชื่อ "Hair Care"

- Name: "Hair Shower"
- Desciption: "Cold hair shower"
- Price: 100 (Provider 1) และ 200 (Provider 2)
- Service Provider: "Provider 1" และ "Provider 2"

*Hint: ดังนั้นต้องสร้าง 2 แถวในตาราง `Service`*


(0.5 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [4]:
prod1 = ServiceProvider.objects.get(name='Provider 1')
prod2 = ServiceProvider.objects.get(name='Provider 2')

serv1 = Service.objects.create(
    service_provider = prod1,
    name='Hair Shower',
    description='Cold hair shower',
    price=100
)

serv2 = Service.objects.create(
    service_provider = prod2,
    name='Hair Shower',
    description='Cold hair shower',
    price=200
)

hair = ServiceCategory.objects.get(name='Hair Care')
hair.services.add(serv1, serv2)

In [5]:
# Check result
cat = ServiceCategory.objects.first()
cat.services.count()

6

1.2 ทำการแก้ไขราคาของ "Hair Shower" ของ "Provider 1" จาก 100 เป็น 300

(0.25 คะแนน)

In [6]:
serv1.price=300
serv1.save()


In [7]:
# Check result
services = Service.objects.filter(name="Hair Shower")
for service in services:
    print(f"Service Name: {service.name}, Provider Name: {service.service_provider.name}, Price: {service.price}")

Service Name: Hair Shower, Provider Name: Provider 2, Price: 200
Service Name: Hair Shower, Provider Name: Provider 1, Price: 300


1.3 ทำการแก้ไขนัดหมาย ณ วันที่ 2024-08-15 โดยเปลี่ยนจากเดิมที่นัดมาให้บริการ "Massage" ของ "Provider 3" เป็น "Hair Shower" ของ "Provider 1" แทน และทำการเลื่อนวันที่นัดหมายจาก 2024-08-15 เป็น 2024-08-18

(0.25 คะแนน)

*หมายเหตุ: จะต้อง get ข้อมูลมาโดยใช้การ query ด้วยชื่อตามที่โจทย์ว่าเท่านั้น ห้ามใช้ id ใน database*

In [13]:
appnt = Appointment.objects.get(appointment_date='2024-08-15')
prov1 = ServiceProvider.objects.get(name='Provider 1')
hairshow = Service.objects.get(name='Hair Shower', service_provider=prov1)

appnt.service = hairshow
appnt.appointment_date = '2024-08-18'
appnt.save()

In [14]:
# Check result
res = Appointment.objects.order_by("-appointment_date").first()
print(f"Appoint Date: {res.appointment_date}, Appoint Time: {res.appointment_time}")
print(f"Service Name: {res.service.name}, Provider Name: {res.service.service_provider.name}, Customer Name: {res.customer.name}")

Appoint Date: 2024-08-18, Appoint Time: 11:30:00
Service Name: Hair Shower, Provider Name: Provider 1, Customer Name: Customer 1


1.4 ทำตามขั้นตอนดังนี้

1. สร้างผู้ให้บริการ (`ServiceProvider`) ชื่อ "Super Provider" ขึ้นมา (email: super@gmail.com, phone: 089-999-9999) และทำการเพิ่มบริการ (`Service`) ให้กับผู้ให้บริการนี้ (เทพมากสามารถทำได้ทุกบริการ) โดยทุกบริการราคาเท่ากับ 500 บาท

(1 คะแนน)

In [24]:
serprov_create = ServiceProvider.objects.create(
    name='Super Provider',
    email='super@gmail.com',
    phone='089-999-9999'
)

supprov = ServiceProvider.objects.get(name='Super Provider')

services1 = Service.objects.create(
    service_provider = supprov,
    name='Hair Shower',
    description='Cold hair shower',
    price=500
)

services2 = Service.objects.create(
    service_provider = supprov,
    name='Haircut',
    description='A simple haircut',
    price=500
)


services3 = Service.objects.create(
    service_provider = supprov,
    name="Manicure",
    description="Nail trimming and polish",
    price=500
)


services4 = Service.objects.create(
    service_provider = supprov,
    name="Massage",
    description="Full body massage",
    price=500
)


hair = ServiceCategory.objects.get(name="Hair Care")
nail = ServiceCategory.objects.get(name="Nail Care")
body = ServiceCategory.objects.get(name="Body Care")

hair.services.add(services1, services2)
nail.services.add(services3)
body.services.add(services4)

In [25]:
# Check result
for s in Service.objects.filter(service_provider__name="Super Provider"):
    print(f"Service Name: {s.name}, Price: {s.price}, Provider: {s.service_provider.name}")

Service Name: Hair Shower, Price: 500, Provider: Super Provider
Service Name: Haircut, Price: 500, Provider: Super Provider
Service Name: Manicure, Price: 500, Provider: Super Provider
Service Name: Massage, Price: 500, Provider: Super Provider


## PART 2 - Making Queries

สำหรับ PART 2 ให้ทำการ reset DB และ import ข้อมูลใน `service.sql` เข้าไปใหม่

2.1 ให้หาจำนวนนัดหมาย ที่นัดหมาย (`Appointment.appointment_date`) ในช่วงวันที่ 2024-08-05 ถึง 2024-08-10 ของ "Service Provider 1"

**Expected Output**

จะต้องมี 3 รายการ

(0.5 คะแนน)

In [14]:
count = Appointment.objects.filter(appointment_date__range=('2024-08-05', '2024-08-10'), service__service_provider_id=1).values('service__service_provider_id').annotate(count=Count('id'))
count[0]['count']

5

2.2 ให้หาว่ามีบริการใด ของผู้ให้บริการไหนบ้าง ที่มีการนัดหมายที่ถูกสร้าง (`Appointment.created_at`) ในช่วงวันที่ 2024-08-01 ถึง 2024-08-02

(0.5 คะแนน)

**Expected Output**

```
Name: Haircut, Provider: Provider 1
Name: Manicure, Provider: Provider 1
Name: Massage, Provider: Provider 1
Name: Manicure, Provider: Provider 2
```

In [69]:
serv = Appointment.objects.filter(created_at__range=('2024-07-31', '2024-08-02')).order_by('service__service_provider__name')

for i in serv:
    print(
        'Name: %s, ' %i.service.name,
        'Provider: %s' %i.service.service_provider.name
    )


Name: Haircut,  Provider: Provider 1
Name: Manicure,  Provider: Provider 1
Name: Massage,  Provider: Provider 1
Name: Manicure,  Provider: Provider 2


2.3 ให้แสดงผลข้อมูลลูกค้า (`Customer`) ดังในตัวอย่าง

```python
[
    {
        "name": "Customer 4",
        "phone": "123-456-7893",
        "appointment_num": 3
    },
    {
        "name": "Customer 2",
        "phone": "123-456-7891",
        "appointment_num": 4
    },
    {
        "name": "Customer 3",
        "phone": "123-456-7892",
        "appointment_num": 3
    },
    {
        "name": "Customer 1",
        "phone": "123-456-7890",
        "appointment_num": 5
    }
]
```

(0.5 คะแนน)

*หมายเหตุ: จะต้องใช้ `annotate()` และ `values()` เท่านั้น การ loop และสร้าง dict เองจะไม่ได้คะแนน*

Hint: อยาก print dictionary สวยๆ ใช้ json.dumps

```python
print(json.dumps(dictionary, indent=4, sort_keys=False))
```

In [34]:
import json
cus = Appointment.objects.values('customer_id').annotate(appointment_num=Count('customer_id')).values('customer__name', 'customer__phone', "appointment_num")
cus
for i in cus:
    print(json.dumps(i, indent=4, sort_keys=False))

{
    "customer__name": "Customer 4",
    "customer__phone": "123-456-7893",
    "appointment_num": 3
}
{
    "customer__name": "Customer 2",
    "customer__phone": "123-456-7891",
    "appointment_num": 4
}
{
    "customer__name": "Customer 1",
    "customer__phone": "123-456-7890",
    "appointment_num": 5
}
{
    "customer__name": "Customer 3",
    "customer__phone": "123-456-7892",
    "appointment_num": 3
}


2.4 ให้แสดงข้อมูลผู้ให้บริการ (`ServiceProvider`) ดังนี้

*หมายเหตุ: Appointment Num หมายถึงจำนวนนัดหมายที่ผู้ให้บริการนั้นมีการได้รับนัด และ Appointed Service หมายถึงบริการที่มีการนัดหมายของผู้ให้บริการนั้นๆ*

(1 คะแนน)

**Expected Output**

```python
Name: Provider 1, Appointment Num: 7, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure, Massage, Massage
Name: Provider 2, Appointment Num: 5, Appointed Services: Haircut, Manicure, Massage, Manicure, Manicure
Name: Provider 3, Appointment Num: 2, Appointed Services: Manicure, Massage
Name: Provider 4, Appointment Num: 1, Appointed Services: Haircut
```

In [92]:
count = Appointment.objects.values('service__service_provider_id__name').annotate(count=Count('id')).order_by('service__service_provider_id__name')
servx= Appointment.objects.values('service__service_provider_id__name')
servx.values('service__name')

<QuerySet [{'service__name': 'Haircut'}, {'service__name': 'Manicure'}, {'service__name': 'Manicure'}, {'service__name': 'Manicure'}, {'service__name': 'Massage'}, {'service__name': 'Massage'}, {'service__name': 'Massage'}, {'service__name': 'Haircut'}, {'service__name': 'Manicure'}, {'service__name': 'Manicure'}, {'service__name': 'Manicure'}, {'service__name': 'Massage'}, {'service__name': 'Manicure'}, {'service__name': 'Massage'}, {'service__name': 'Haircut'}]>

In [86]:
for i in count:
    print(
        'Name: %s,' %i['service__service_provider_id__name'],
        'Appointment Num: %d,' %i['count'],
#         'Appointed Services: %s' %i['service_id__name']
    )

Name: Provider 1, Appointment Num: 7,
Name: Provider 2, Appointment Num: 5,
Name: Provider 3, Appointment Num: 2,
Name: Provider 4, Appointment Num: 1,


2.5 ให้หาว่าบริการ (`Service`) ไหนของผู้บริการไหนได้รับความนิยมสูงสุด (มีจำนวนนัดหมายมากที่สุด) และ นิยมน้อยที่สุด (มีจำนวนนัดหมายน้อยที่สุด)

(0.5 คะแนน)

**Expected Output**

```python
Name: Manicure, Provider: Provider 1, Appointment Num: 3
Name: Massage, Provider: Provider 1, Appointment Num: 3
Name: Manicure, Provider: Provider 2, Appointment Num: 3
Name: Massage, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 1, Appointment Num: 1
Name: Massage, Provider: Provider 2, Appointment Num: 1
Name: Manicure, Provider: Provider 3, Appointment Num: 1
Name: Haircut, Provider: Provider 4, Appointment Num: 1
Name: Haircut, Provider: Provider 2, Appointment Num: 1
Name: Massage, Provider: Provider 4, Appointment Num: 0
Name: Manicure, Provider: Provider 4, Appointment Num: 0
Name: Haircut, Provider: Provider 3, Appointment Num: 0
```

In [107]:
pop = Appointment.objects.values('service__name', 'service__service_provider__name').annotate(count=Count('id')).order_by('-count')
pop

<QuerySet [{'service__name': 'Manicure', 'service__service_provider__name': 'Provider 2', 'count': 3}, {'service__name': 'Massage', 'service__service_provider__name': 'Provider 1', 'count': 3}, {'service__name': 'Manicure', 'service__service_provider__name': 'Provider 1', 'count': 3}, {'service__name': 'Massage', 'service__service_provider__name': 'Provider 3', 'count': 1}, {'service__name': 'Haircut', 'service__service_provider__name': 'Provider 4', 'count': 1}, {'service__name': 'Manicure', 'service__service_provider__name': 'Provider 3', 'count': 1}, {'service__name': 'Haircut', 'service__service_provider__name': 'Provider 2', 'count': 1}, {'service__name': 'Haircut', 'service__service_provider__name': 'Provider 1', 'count': 1}, {'service__name': 'Massage', 'service__service_provider__name': 'Provider 2', 'count': 1}]>

In [108]:
for i in pop:
    print(
        'Name: %s,' %i['service__name'],
        'Provider: %s' %i['service__service_provider__name'],
        'Appointment Num: %d' %i['count']
    )

Name: Manicure, Provider: Provider 2 Appointment Num: 3
Name: Massage, Provider: Provider 1 Appointment Num: 3
Name: Manicure, Provider: Provider 1 Appointment Num: 3
Name: Massage, Provider: Provider 3 Appointment Num: 1
Name: Haircut, Provider: Provider 4 Appointment Num: 1
Name: Manicure, Provider: Provider 3 Appointment Num: 1
Name: Haircut, Provider: Provider 2 Appointment Num: 1
Name: Haircut, Provider: Provider 1 Appointment Num: 1
Name: Massage, Provider: Provider 2 Appointment Num: 1


2.6 ให้ดึงข้อมูลหมวดหมู่บริการโดยแปลงชื่อบริการ (`Service.name`) เป็นตัวใหญ่ (uppercase) และราคา (`Service.price`) เป็น 2 เท่า และเรียงลำดับตามราคา

(0.5 คะแนน)

**Expected Output**

```python
Name: HAIRCUT, Provider: Provider 1, New Price: $40
Name: MANICURE, Provider: Provider 1, New Price: $60
Name: MASSAGE, Provider: Provider 1, New Price: $200
Name: HAIRCUT, Provider: Provider 2, New Price: $40
Name: MANICURE, Provider: Provider 2, New Price: $60
Name: MASSAGE, Provider: Provider 2, New Price: $200
Name: HAIRCUT, Provider: Provider 3, New Price: $40
Name: MANICURE, Provider: Provider 3, New Price: $60
Name: MASSAGE, Provider: Provider 3, New Price: $200
Name: HAIRCUT, Provider: Provider 4, New Price: $40
Name: MANICURE, Provider: Provider 4, New Price: $60
Name: MASSAGE, Provider: Provider 4, New Price: $200
```

In [59]:
# CODE HERE
